In [ ]:
import re
import string
import nltk
from nltk.tokenize import TweetTokenizer, TreebankWordDetokenizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
# # Download required NLTK datasets
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize tools
tweet_tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

# Combine custom stopwords with NLTK's stopwords

stopwords_set = set(nltk_stopwords.words('english'))

def normalize(text):
    # Normalize text
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text= emoji_pattern.sub(r'', text)

    text = re.sub(r'@[A-Za-z0-9_]+', 'USER', text)  # Replace @username
    text = re.sub(r'https?://[A-Za-z0-9./]+', 'URL', text)  # Replace URLs
    text = re.sub(r'\n', ' ', text)  # Remove new line breaks
    text = "".join([char for char in text if char not in string.punctuation])  # Remove punctuation
    text = re.sub(' +', ' ', text.strip())  # Remove extra spaces
    text = re.sub(r'\.{3,}', '', text)  # Loại bỏ dấu ba chấm (...)
    text = re.sub(r'([!?.]){2,}', r'\1', text)  # Loại bỏ dấu câu lặp
    text = re.sub(r'url','',text) # Loại bỏ url
    text = re.sub(r'\b(\S*?)(.)\2{2,}\b', r'\1\2', text)  # Loại bỏ từ lặp
    text = re.sub(r'([A-Za-z0-9])\1+', r'\1\1', text)  # Loại bỏ ký tự lặp
    
    # Remove emoticons like :), :v, etc.
    text = re.sub(r'[:;=]-?[)(DPpOo/\|\\]', '', text)  # Basic emoticons
    text = re.sub(r'[xX][Dd]', '', text)  # XD type emoticons
    text = re.sub(r'\^_?\^', '', text)  # ^^ type emoticons

    return text


def clean(text):
    # Clean text
    text = text.lower()
    text = normalize(text)  # Normalize first
    tokens = tweet_tokenizer.tokenize(text)
    tokens = [t for t in tokens if t not in stopwords_set]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(x) for x in tokens]  # Lemmatize

    # Convert back to string
    return TreebankWordDetokenizer().detokenize(tokens)

import json

# Giả sử hàm clean đã được định nghĩa như trước đây

def process_json(json_data):
    for user_data in json_data:
        if 'tweets' in user_data:
            for tweet in user_data['tweets']:
                if 'tweet_content' in tweet:
                    tweet['tweet_content'] = clean(tweet['tweet_content'])

        # Làm sạch phần giới thiệu của người dùng
        user_intro = user_data['user'].get('intro', '')
        user_data['user']['intro'] = clean(user_intro)

    return json_data

# Đọc file JSON
with open('Data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

def write_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Specify the file path for the new JSON file
output_file_path = 'Data1.json'

# Áp dụng hàm clean
cleaned_data = process_json(data)
write_json(cleaned_data, output_file_path)

Gắn project tag cho từng user từ Data của centic

In [ ]:
import json

# Function to read data from a JSON file
def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Function to write data to a JSON file
def write_json_file(data, file_path):
    with open(file_path, 'w', encoding ='utf-8') as file:
        json.dump(data, file, indent=4)

# Đọc dữ liệu từ các tệp JSON
projects_data = read_json_file('project_list.json')
project_followers_data = read_json_file('project_followers_fixed.json')
followers_data = read_json_file('Bigdata.json')

# Tạo từ điển để ánh xạ từ link sang tên project và từ tên project sang category
link_to_project_name = {project['link']: project['name'] for project in projects_data}
project_name_to_category = {project['name']: project['category'] for project in projects_data}

# Hàm để trả về danh sách các tên project mà một follower theo dõi, loại bỏ trùng lặp
def get_unique_followed_projects(follower_username):
    followed_projects = set()
    for project_follower in project_followers_data:
        if follower_username in [follower['username'] for follower in project_follower['followers']]:
            project_name = link_to_project_name.get(project_follower['link'], None)
            if project_name:
                followed_projects.add(project_name)
    return list(followed_projects)

# Hàm để lấy thông tin project-following và category tương ứng
def get_projects_and_categories(followed_project_names):
    projects_and_categories = []
    for project_name in followed_project_names:
        category = project_name_to_category.get(project_name, "Not Found")
        projects_and_categories.append({'project': project_name, 'category': category})
    return projects_and_categories

# Cập nhật dữ liệu followers với danh sách các project theo dõi duy nhất và category tương ứng
for follower in followers_data:
    follower_username = follower['user']['username']
    unique_projects = get_unique_followed_projects(follower_username)
    projects_categories = get_projects_and_categories(unique_projects)
    follower['projects_following'] = projects_categories

# Ghi dữ liệu đã cập nhật vào tệp JSON mới
write_json_file(followers_data, 'updated_followers_with_projects.json') // Data.json